# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>


# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. La persona interesada más sobre ellos <https://huggingface.co/>. 
Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

En lo que resta de este notebook se estarán viendo como usar los modelos, y más precisamente los embeddings para realizar tareas de recuperación de información.


In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
using SimilaritySearch, LinearAlgebra, HypertextLiteral, CodecZlib, JSON, Random, StatsBase, DataFrames, JLD2
using Downloads: download

# Using Transformers as encoders
## Using SentenceBert

In [3]:
using PyCall # note that I already installed sentence-tranformers package and configured PyCall
pushfirst!(pyimport("sys")."path", "") # this is necessary to load modules in the current path
sbert = pyimport("sbertencoder")

PyObject <module 'sbertencoder' from '/home/sadit/Cursos/IR-2024/Unidades/sbertencoder.py'>

In [4]:
langmodel = sbert.TextEncoder()

PyObject <sbertencoder.TextEncoder object at 0x7fcc98622390>

In [5]:
function encode_text(model, text::AbstractString; normalize=true)
    e = model.encode(text)
    normalize && normalize!(e)
    e
end

function encode_corpus(model, corpus::AbstractVector; normalize=true)
    n = length(corpus)
    e = model.encode(corpus)
    E = permutedims(e)
    @assert size(E, 1) == 384 # only for sbert
    if normalize
        for c in eachcol(E)
            normalize!(c)
        end
    end

    E
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db)
    minrecall = MinRecall(recall)
    ctx = SearchGraphContext(;
        hyperparameters_callback=OptimizeParameters(minrecall)
    )
    index!(index, ctx)
    optimize_index!(index, ctx, minrecall)
    index
end

create_index (generic function with 1 method)

In [6]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
train, test = DataFrame.(read_news())
first(train, 3)

<h1>Cargando el corpus</h1>

Row,text,labels
,String,String
1,RT @SanGonAzc: @RadioIndeMexico @FiscaliaNL @FGRMexico @lopezobrador_ @AldoFasciZ @RicardoMeb se van a seguir haciendo tontos?? #debaniesc…,AdriDelgadoRuiz
2,"“El periodismo serio y responsable es una herramienta esencial para salvaguardar y fortalecer el sistema democrático en México y el mundo, y hoy se debe fortalecer esa profesión”, afirmó el periodista y escritor, Harris Whitbeck @harriswhitbeck\n#ElDedoEnLaLLaga #HeraldoRadio",AdriDelgadoRuiz
3,"""...Sí esperamos que se aumente la tasa en 25 puntos base en las siguientes dos reuniones y continuar así durante el primer trimestre del 2022”: @raguilar_abe",AdriDelgadoRuiz


In [7]:
function load_or_compute_embeddings(langmodel, corpus, h5filename; normalize=false)
    if !isfile(h5filename)
        @time X = encode_corpus(langmodel, test.text)
        jldsave(h5filename; emb=X)
    else
        X = jldopen(h5filename) do f
            f["emb"]
        end
    end

    if normalize
        for c in eachcol(X)
            normalize!(c)
        end
    end

    X
end

load_or_compute_embeddings (generic function with 1 method)

In [8]:
trainfile = "data/spanish-twitter-news-and-opinions-top25-68.train--sbert-multi-L12-v2.h5"
testfile = "data/spanish-twitter-news-and-opinions-top25-68.test--sbert-multi-L12-v2.h5"

Xtrain = load_or_compute_embeddings(langmodel, train.text, trainfile)
Xtest = load_or_compute_embeddings(langmodel, test.text, testfile)

size(Xtrain), size(Xtest)

((384, 30244), (384, 3361))

### Se crea el índice métrico

In [9]:
@time index = create_index(MatrixDatabase(Xtrain), 0.95)

append_items! sp=29555, ep=29811, n=29811, BeamSearch
  bsize: Int32 26
  Δ: Float32 1.0875f0
  maxvisits: Int64 2620
, 2024-01-10T13:20:37.024


 11.006896 seconds (5.13 M allocations: 380.170 MiB, 1.96% gc time, 42.53% compilation time: <1% of which was recompilation)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  adj: SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}
  hints: Array{Int32}((102,)) Int32[60, 194, 208, 223, 344, 365, 377, 454, 484, 497  …  2585, 2629, 2667, 2668, 2673, 2675, 2682, 2711, 2798, 2799]
  search_algo: BeamSearch
  len: Base.RefValue{Int64}


### Búsqueda de todos los vecinos cercanos de test en train, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad

In [10]:
let k = 32
    @time Iknns, Idists = searchbatch(index, MatrixDatabase(Xtest), k)
    exs = ExhaustiveSearch(; dist=distance(index), db=database(index))
    @time Eknns, Edists = searchbatch(exs, MatrixDatabase(Xtest), k)
    @show macrorecall(Eknns, Iknns)
end

  0.385207 seconds (444.71 k allocations: 30.422 MiB, 89.78% compilation time)
  0.758840 seconds (246.52 k allocations: 17.090 MiB, 4.68% gc time, 35.65% compilation time)
macrorecall(Eknns, Iknns) = 0.8539404195180006


0.8539404195180006

### Búsqueda y presentación de los resultados

In [11]:
#Eindex = ExhaustiveSearch(; dist=distance(index), db=database(index))
E = (; langmodel, index, data=train); # para simplificar algunas funciones

In [12]:
function search_and_display(E, qtext, k)
    res = KnnResult(k)
    q = encode_text(E.langmodel, qtext)

    @time search(index, q, res)
    
    L = []
    for (j, p) in enumerate(res)
        id, d = p.id, p.weight
        text, label = E.data.text[id], E.data.labels[id]
        push!(L, @htl "<tr><td>$j</td><td>$(id)</td><td>$(round(d, digits=3))</td> <td>$(label)</td><td> $(text)</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [13]:
display(@htl "<h1>Ejemplos de búsqueda</h1>")
#search_and_display(E, E.data.corpus[1], 7)
search_and_display(E, "busqueda de personas desaparecidas", 7)
search_and_display(E, "sobre el covid 19 y el coronavirus", 7)
search_and_display(E, "arte y cine", 7)
search_and_display(E, "sobre el narcotráfico", 7)

<h1>Ejemplos de búsqueda</h1>

  0.157288 seconds (99.95 k allocations: 6.366 MiB, 99.85% compilation time)


1,9927,0.284,UniNoticias,🎥 Esta familia pide ayuda para encontrar a una mujer de 60 años reportada como desaparecida. https://t.co/XkV0npQ9BS
2,12201,0.3,Milenio,"▶️ Continúa búsqueda de Debahni Escobar, joven de 18 años desaparecida desde el 8 de abril en Nuevo León En la búsqueda participaron cerca de 50 personas entre familiares, amigos, colectivos y sociedad civil 📺 #Milenio19h con @pedrogamboamr https://t.co/vHAaTuznT6"
3,14255,0.327,julioastillero,🚢 #EnVivo en #AstilleroInforma El tema de desaparición forzada se ha convertido en un dolor para la sociedad. Todas las personas tienen el derecho a ser buscadas y encontradas. Es el momento de echar a andar esto a rango constitucional: @Omarel44 https://t.co/qe3FatOwmv https://t.co/wuOVVFovff
4,24109,0.343,abc_es,Vidas sin rastro: la angustia de miles de búsquedas https://t.co/1YNePO0dTJ ✍ Un reportaje de Beatriz L. Echazarreta
5,24560,0.355,epigmenioibarra,"RT @jenarovillamil: Ay, diputada @Mzavalagc, qué tal si revisamos las listas de desaparecidos en la lista del sexenio calderonista"
6,1281,0.373,Milenio,"▶️ ""Las desapariciones no han frenado y nos habla de un problema presente y una deuda con las víctimas"": Sofía de Robina, abogada del Área Internacional del Centro PRODH México llegó a las 100 mil personas desaparecidas 📺 La entrevista con @elisaalanis #ElisaEnMILENIO https://t.co/3FRfarsXyh"
7,11112,0.376,julioastillero,"Desaparecida, en #Apodaca #NuevoLeón :"


  0.000111 seconds (5 allocations: 192 bytes)


1,26658,0.211,abc_es,Adiós a la mascarilla en exteriores: así quedan las restricciones por el coronavirus por comunidades https://t.co/w9sIL69Xxy
2,21235,0.213,abc_es,Qué es la variante Delta del coronavirus https://t.co/oTKQRropai
3,29257,0.216,CNNEE,Así es como surgen las variantes del nuevo coronavirus. https://t.co/qH1ggXs3KH
4,20772,0.222,abc_es,Así quedan las restricciones por el coronavirus por comunidades https://t.co/1AC0unePne
5,17655,0.23,RicardoAlemanMx,"El subsecretario de Salud, López-Gatell, confirmó el primer caso de coronavirus covid-19 con ómicron en México https://t.co/iTaO3k7DfQ"
6,4076,0.233,Radio_Formula,La altamente transmisible variante ómicron ha provocado un alza de casos de coronavirus en todo el mundo. https://t.co/N8LHD2DGFO
7,8031,0.233,PublimetroCol,Este viernes solo hay 20.221 casos activos de coronavirus en el país https://t.co/jBj5pD28Vn


  0.000133 seconds (5 allocations: 192 bytes)


1,6010,0.366,el_pais,"RT @elpais_cultura: El cineasta estrena en Lisboa ‘Andy’, su primera obra de teatro, dedicada al visionario artista del ‘pop-art’ con el qu…"
2,2188,0.391,el_pais,"""El desprecio contra la gente del cine es algo histórico, somos el oficio de 'los rojos'. Es muy grave que desde instituciones que manejan presupuestos públicos pueda despreciarse un campo de expresión artística"", lamenta Alberto San Juan https://t.co/pKVnkddEQj"
3,30110,0.415,El_Universal_Mx,RT @Univ_Cultura: ¿Cómo es que terminaste teniendo “perrhijos” y “gathijos”? Esta exposición te lo explica a través de la historia del arte…
4,21570,0.417,PublimetroCol,"Hablamos en exclusiva con dos de las mentes creativas detrás de esta película que promete enamorar a muchos al son de música, tradición, culturas y diversión. https://t.co/G0sT9XEYuK"
5,18242,0.418,EFEnoticias,"ENTREVISTA | Pedro Halffter: La convivencia con el arte debe ser en vivo, no en la pantalla. Por Sergio Jiménez Foronda https://t.co/8mZg1cEkhl https://t.co/aHH0z4Y612"
6,11035,0.419,PublimetroMX,"#Entrevista | @tokischa_ :“El arte en general es libre para expresar lo que uno sienta y se imagine"". https://t.co/MxwFvkmZQd"
7,1597,0.422,teleSURtv,"#ENVIDEO | El #ArteContemporáneo tiene diferentes representaciones, entre ellas el #LandArt Las piezas de arte de esta corriente se las realiza en el entorno natural Conoce más sobre esta puesta en escena en el siguiente vídeo https://t.co/alKzXTItPM"


  0.000124 seconds (5 allocations: 192 bytes)


1,15440,0.289,abc_es,"Drogas https://t.co/WVZaCfEuUP ✍ 'Tveo', por @rosabelmonte"
2,26384,0.336,PublimetroChile,"Tras los buenos resultados del Estudio Nacional de Drogas en Población General, publicados el...👇 https://t.co/I4HzdWR9Uj"
3,28700,0.337,RicardoAlemanMx,RT @ContrapesoC: Este es el pacto del narco y Cuauhtémoc Blanco. ¿Alguna duda del narcoestado? https://t.co/qDpAwTKcHP
4,3930,0.339,AristeguiOnline,La mayoría de los cárteles se ha centrado en el narcotráfico... hasta que en 2006 comenzaron a involucrarse en otras empresas criminales. https://t.co/LEj1AA0Dky
5,22011,0.341,AristeguiOnline,El comercio ilegal de drogas se digitaliza con la pandemia https://t.co/dzZtvcjBAA https://t.co/hrG2x5a5xs
6,26428,0.344,RicardoAlemanMx,RT @RicardoAlemanMx: “Daños colaterales”: Patricia Armendáriz sobre “pérdidas” por desabasto de medicamentos https://t.co/CyzKK301z3
7,23204,0.347,bbcmundo,"""Los recolectores de cocaína"" de Rotterdam: la vida en los sombras de los jóvenes que descargan la droga que viaja de América Latina a Europa https://t.co/1NJXa7kpgs"


In [14]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el train)</h1>")

for i in 1:3
    for qid in rand(1:length(train.text))
        search_and_display(E, train.text[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el train)</h1>

  0.000141 seconds (5 allocations: 192 bytes)


1,24348,-0.0,EFEnoticias,"El secretario estadounidense de Seguridad Nacional, Alejandro Mayorkas, fue interpelado en el Senado acerca de los migrantes que cruzan la frontera ilegalmente y que tras una breve detención son dejados en libertad con cita para una audiencia. https://t.co/TfUAKRySHl"
2,28781,0.251,UniNoticias,"➡️ El secretario de Seguridad Nacional, Alejandro Mayorkas, reconoce que el sistema migratorio está ""roto"". ➡️ El funcionario niega que se esté separando a niños de sus familias en la frontera como ocurrió durante el gobierno anterior. Informa @pablogatoDC https://t.co/rCDJa3mbyv"
3,5532,0.31,UniNoticias,"🎥 ¿A quiénes beneficia este anuncio del gobierno? El abogado experto en inmigración, @HernandezGlobal, explica los detalles 👇. ✅ Sigue esta y más noticias las 24 horas en @PrendeTV. https://t.co/FmJO38p2VC"
4,11157,0.323,CNNEE,El secretario de Seguridad Nacional de EE.UU. advierte a migrantes de la peligrosidad de su viaje por la frontera tras el incidente que dejó 53 muertos https://t.co/DwqLnVtt3c
5,8995,0.326,UniNoticias,"🎥 El secretario de Seguridad Nacional, @AliMayorkas, visitó la frontera y pidió paciencia, indicando que ya hay un plan en ejecución para detener el flujo migratorio. Por: @cobostv. https://t.co/2nkeTJ5dgV"
6,507,0.339,UniNoticias,“¿Puedo pedir la liberación de un familiar detenido en una cárcel de ICE?” Respondemos tus preguntas de inmigración. https://t.co/7QOouvsSm1
7,10664,0.341,AristeguiOnline,"#AdelantoEditorial | 'Indocumentadas', un thriller sobre migración en la frontera México-EU 👉https://t.co/IkokA1zQeH https://t.co/T132aHwUwl"


  0.000113 seconds (5 allocations: 192 bytes)


1,23478,-0.0,epigmenioibarra,RT @Magdale26991438: 💪
2,17016,0.156,epigmenioibarra,RT @alvaroa36301590: 🤢🤢🤢🤢🤢🤮🤮🤮🤮🤮🤮
3,15381,0.202,epigmenioibarra,RT @jesusca66564913: Va
4,13279,0.205,epigmenioibarra,RT @_Jaramilloed_: 👇🏼👇🏼👇🏼
5,25026,0.21,epigmenioibarra,RT @Alejand31822792: Que siga la 4T
6,30048,0.213,RicardoAlemanMx,RT @PabloHiriart: 👇
7,2802,0.22,epigmenioibarra,RT @ANGELICASALAZA7: #Presente 🙋✋ #PaseDeLista1al43x43


  0.000165 seconds (5 allocations: 192 bytes)


1,28652,0.0,RicardoAlemanMx,#Video📹 | AMLO exhibe foto de Calderón donde sale su actual jefe de asesores de su gobierno https://t.co/6dqcFepmgl https://t.co/yu3BMgQJ6Y
2,6975,0.131,El_Universal_Mx,"#VIDEO 📹 Felipe Calderón responde a AMLO: ""El dinero se lo mandan a otro presidente"" https://t.co/b5V2hpz6Y6"
3,18286,0.178,RicardoAlemanMx,AMLO arremetió contra Calderón https://t.co/h6eMNroqT1
4,29914,0.205,SinEmbargoMX,"Felipe Calderón se dice perseguido por el Gobierno de AMLO. Es autoritarismo, dice https://t.co/ur3Fcw0rsF https://t.co/AFolNxXhkh"
5,23024,0.212,El_Universal_Mx,"#LaMañaneraDeAMLO AMLO: Felipe Calderón vino vestido de militar a Michoacán a declarar la guerra, a pegarle un garrotazo a lo tonto al avispero, sin un plan https://t.co/SIgxpzHFm2"
6,434,0.248,julioastillero,"🚢 #EnVivo en #AstilleroInforma El que calla otorga, entonces, todo lo que se ha publicado pasa a ser parte de una verdad mientras Felipe Calderón no de una explicación. Qué bueno que AMLO lo instó para que aclare su relación con García Luna: @RRavelo27 https://t.co/KeZFk1RS8H https://t.co/eSt1gcKPVg"
7,26683,0.252,El_Universal_Mx,"Califican como ""mamarracho"" al expresidente Felipe Calderón tras unirse a los festejos del piloto Sergio ""Checo"" Pérez https://t.co/6kNFlKuUOn"


In [15]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>")

for i in 1:3
    for qid in rand(1:length(test.text))
        search_and_display(E, test.text[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>

  0.000151 seconds (5 allocations: 192 bytes)


1,28631,0.151,CNNEE,El ejército ucraniano dice que un tanque ruso fue destruido en el oeste de Kyiv https://t.co/JzA6yKRenH
2,4848,0.18,TelemundoNews,Rusia asegura que destruyó con misiles de alta precisión las naves de una fábrica de tanques en la capital ucraniana. https://t.co/QDVOQvny5s
3,18020,0.234,EFEnoticias,#EFEURGENTE | Ucrania denuncia la entrada de columnas de tanques rusos en Lugansk. https://t.co/M5M08s5MIU
4,6876,0.268,LaRazon_mx,#Ucrania asegura haber matado a más de mil soldados rusos y capturado a 300 https://t.co/TeIadkwvnx https://t.co/9oMlPSFIDz
5,16744,0.281,abc_es,#LoMásLeído ¿Por qué Rusia ha comenzado a invadir Ucrania? https://t.co/XkZyotlVv3
6,23397,0.282,NTN24,"Según Rusia, sus tropas destruyeron 44 “instalaciones militares” en Ucrania https://t.co/laZ9O024Qo"
7,1552,0.301,NTN24,"Volodymyr Zelensky, afirmó que ""alrededor de 1.300 militares"" ucranianos murieron desde el inicio de la invasión rusa https://t.co/LeQ9XYLbMB"


  0.000159 seconds (5 allocations: 192 bytes)


1,16850,0.008,bbcmundo,Rogel Lázaro Aguilera-Mederos chocó el camión de carga de madera que conducía contra varios carros. https://t.co/mgrBl7upI5
2,11809,0.373,PublimetroChile,El conductor que provocó el accidente manejó por 10km en contra del tránsito impactando con una camioneta un...👇 https://t.co/721jjKQW7X
3,28013,0.394,RicardoAlemanMx,¡Esto fue lo que le ocurrió al conductor! https://t.co/XaqEMx8kDC
4,14710,0.414,abc_es,Los carburantes hacen un roto en el bolsillo de los conductores en la Operación Salida de agosto https://t.co/r20ZofyMwn
5,12150,0.417,SinEmbargoMX,VIDEO FUERTE: Tanque aplasta totalmente un auto en Kiev; conductor sobrevive https://t.co/Wz3PpJXaDs https://t.co/UW32MQLhlo
6,7223,0.423,El_Universal_Mx,"RT @elgmx: 🚨 Imágenes del accidente de este viernes, donde un tráiler sin frenos chocó contra un camión de pasajeros, lo que dejó cuatro pe…"
7,19057,0.426,PublimetroChile,"El conductor del vehículo, que iba bajo la influencia del alcohol, colisionó contra el muro de una vivienda👇 https://t.co/5tiyIZsXIq"


  0.000152 seconds (5 allocations: 192 bytes)


1,5630,0.382,PublimetroChile,La abogada aseguró que aún “no hay medida suficiente” para revertir la situación.👇 https://t.co/qXCLJ3tVMh
2,9606,0.386,azucenau,"""Lo que hay que determinar para mi cliente es simplemente justicia"": Javier Schütte Ricaud, abogado de Ainara #AzucenaxFórmula"
3,28839,0.406,Reforma,"Juez dictó prisión preventiva oficiosa al empresario #BaltazarReséndez, lo que implica que permanecerá en Penal del Altiplano hasta que se defina su situación jurídica por el caso de .@fgcabezadevaca https://t.co/X2wewRzsId"
4,7741,0.416,epigmenioibarra,RT @ChairoBatman: @Lubruixa @epigmenioibarra #PaseDeLista1al43 #PaseDeLista1al43x43 con @epigmenioibarra que haya justicia en el caso de…
5,15258,0.422,azucenau,"""Se seguirá exigiendo #JusticiaparaAinara"": Javier Schütte Ricaud, abogado de Ainara #AzucenaxFórmula"
6,5324,0.443,Reforma,"#Entérate Una jueza federal admitió a trámite concurso mercantil de @ALTANMx, operadora de la #RedCompartida, luego de que la empresa presentó documentos que no había anexado en su demanda original https://t.co/crjSvSe93B"
7,27612,0.445,AristeguiOnline,Mesa de Análisis: Se nos están cargando temas jurídicos de enorme importancia: Cossío | Video https://t.co/Mc68IG8iCc https://t.co/0G84TNVIBR


# Actividades
- Reproduzca el ejercicio de este notebook
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Utilece un extracto de 30 mensajes del test y compare el SBERT (usado en este notebook) contra  el modelo BERT `https://huggingface.co/bert-base-multilingual-cased` y BETO `https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased`; adicionalmente incluya a los modelos tradicionales basados en vocabularios e índices invertidos.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas solicitadas. Discuta sus resultados. Finalice el reporte con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.
- [RG2020] Reimers, N., & Gurevych, I. (2020). Making monolingual sentence embeddings multilingual using knowledge distillation. arXiv preprint arXiv:2004.09813.
- [CCFHKP2023] Cañete, J., Chaperon, G., Fuentes, R., Ho, J. H., Kang, H., & Pérez, J. (2023). Spanish pre-trained bert model and evaluation data. arXiv preprint arXiv:2308.02976.
